## 🎶  [Day 16](https://adventofcode.com/2018/day/18)

In [0]:
import ast 

def parse_inputs(inputs):
  """Parse the two parts in the input file"""
  inputs = inputs.split('\n\n')
  clauses = inputs[:-2]
  clauses = [c.splitlines() for c in clauses]
  clauses = [(int(b[:-6]), int(b[-5]), int(b[-3]), int(b[-1]),
              ast.literal_eval(a.replace('Before: ', '')),
              ast.literal_eval(c.replace('After:  ', '')))
             for a, b, c in clauses]
  test = [list(map(int, line.split())) for line in inputs[-1].split('\n')[:-1]]
  return clauses, test

In [0]:
from functools import partial

"""Defines all instructions"""
def apply_add(a, b, c, before, register=0):
  before[c] = before[a] + (before[b] if register else b)
  return before

def apply_mul(a, b, c, before, register=0):
  before[c] = before[a] * (before[b] if register else b)
  return before

def apply_ban(a, b, c, before, register=0):
  before[c] = before[a] & (before[b] if register else b)
  return before

def apply_bor(a, b, c, before, register=0):
  before[c] = before[a] | (before[b] if register else b)
  return before

def apply_set(a, b, c, before, register=0):
  before[c] = before[a] if register else a
  return before

def apply_gt(a, b, c, before, register=0):
  if register == 0:
    before[c] = a > before[b]
  elif register == 1:
    before[c] = before[a] > b
  else:
    before[c] = before[a] > before[b]    
  return before

def apply_eq(a, b, c, before, register=0):
  if register == 0:
    before[c] = a == before[b]
  elif register == 1:
    before[c] = before[a] == b
  else:
    before[c] = before[a] == before[b]    
  return before

"""Gather all instructions"""
operations = [('addr', partial(apply_add, register=1)), ('addi', partial(apply_add, register=0)),
              ('mulr', partial(apply_mul, register=1)), ('muli', partial(apply_mul, register=0)),
              ('banr', partial(apply_ban, register=1)), ('bani', partial(apply_ban, register=0)),
              ('borr', partial(apply_bor, register=1)), ('bori', partial(apply_bor, register=0)),
              ('setr', partial(apply_set, register=1)), ('seti', partial(apply_set, register=0)),
              ('gtir', partial(apply_gt, register=0)), ('gtri', partial(apply_gt, register=1)),
              ('gtrr', partial(apply_gt, register=2)),
              ('eqir', partial(apply_eq, register=0)), ('eqri', partial(apply_eq, register=1)),
              ('eqrr', partial(apply_eq, register=2))]

In [0]:
def behaves_like(a, b, c, before, after, op):
  """Behaviour test"""
  expected = before.copy()
  op(a, b, c, expected)
  return after == expected

def number_behaviours(inputs, threshold=3):
  """Returns the number of samples with `threshold` or more behaviours"""
  clauses, _ = parse_inputs(inputs)
  behave_ops = [partial(behaves_like, op=op) for _, op in operations]
  i = 0
  for opcode, a, b, c, before, after in clauses:
    num_ops = sum([f(a, b, c, before, after) for f in behave_ops])
    if num_ops >= threshold:
      i += 1
  return i    

def get_exact_behaviour(clauses, threshold=3):
  """Returns the number of samples with `threshold` or more behaviours"""
  num_ops = len(operations)
  
  # Work out assignment possibilities
  ops_matches = [[True for _ in range(num_ops)] for _ in range(num_ops)]
  behave_ops = [partial(behaves_like, op=op) for _, op in operations]
  for opcode, a, b, c, before, after in clauses:
    for matchcode, test in enumerate(behave_ops): 
      ops_matches[opcode][matchcode] = test(a, b, c, before, after) and ops_matches[opcode][matchcode]
      
  # Greedy assignment
  assignment = [0] * num_ops
  for i in range(num_ops):
    opcode = min(range(num_ops), key=lambda x: sum(ops_matches[x]))
    assert sum(ops_matches[opcode]) == 1
    match = ops_matches[opcode].index(True)
    assignment[opcode] = match
    # Update for next
    for j in range(num_ops):
      ops_matches[j][match] = False
    ops_matches[opcode] = [True] * num_ops
  return assignment

def apply_exact_behaviour(inputs, verbose=True):
  """Find the correct assignment from part 1 and apply it to part 2"""
  clauses, test = parse_inputs(inputs)
  assignment = get_exact_behaviour(clauses)
  if verbose:
    print('Matched:')
    for opcode, matchcode in enumerate(assignment):
      print('  Opcode %d matched to %s' % (opcode, operations[matchcode][0]))
  before = [0, 0, 0, 0]
  for opcode, a, b, c in test:
    _, op = operations[assignment[opcode]]
    op(a, b, c, before)
  return before

In [6]:
!wget -q -O day14.txt "https://docs.google.com/uc?export=download&id=1s_UXgKu6J-WXhIY6H76pUnaTtpaa9ZM6"
with open("day14.txt", 'r') as f: 
  inputs = f.read()
  
print('Number of sample with >= 3 behaviours:', number_behaviours(inputs))
print()
print('Final Register after applying the input program:', apply_exact_behaviour(inputs, verbose=True))

Number of sample with >= 3 behaviours: 509

Matched:
  Opcode 0 matched to setr
  Opcode 1 matched to eqrr
  Opcode 2 matched to gtri
  Opcode 3 matched to muli
  Opcode 4 matched to eqir
  Opcode 5 matched to borr
  Opcode 6 matched to bori
  Opcode 7 matched to mulr
  Opcode 8 matched to gtrr
  Opcode 9 matched to seti
  Opcode 10 matched to banr
  Opcode 11 matched to eqri
  Opcode 12 matched to addr
  Opcode 13 matched to gtir
  Opcode 14 matched to addi
  Opcode 15 matched to bani
Final Register after applying the input program: [496, 2, 3, 496]
